In [1]:
import os
import gzip
import requests
import ast
import tqdm
import json

In [2]:
def optional_make_dir(directory):
    if not os.path.isdir(directory):
        os.mkdir(directory)
    
    return None

def download_json_gzip(url, save_directory=''):
#     url = 'http://cseweb.ucsd.edu/~wckang/steam_reviews.json.gz'
#     url = 'http://cseweb.ucsd.edu/~wckang/steam_games.json.gz'
    filename = url.split("/")[-1]
    filename = os.path.join(save_directory, filename)
    if not os.path.isfile(filename):
        with open(filename, "wb") as f:
            r = requests.get(url)
            f.write(r.content)
    
    return None

def gzip_to_json(gzip_filename, directory):
    gzip_filename = os.path.join(directory, gzip_filename)
    json_filename = gzip_filename.replace('.gz', '')
    
    if not os.path.isfile(json_filename):
        print('stage 1: reading gzip (est: 3min)')
        with gzip.open(gzip_filename, 'rb') as file:
            data = file.read()

        print('stage 2: decoding (est: 1min)')
        data = data.decode('utf-8')
        print('stage 3: cleaning (est: 1min)')
        if data.endswith('\n'):
            data = data[:-1]
        print('stage 4: segmenting (est: 1min)')
        data = data.split('\n')
        print('stage 5: interpreting (est: 25min)')
        data = [ast.literal_eval(dict_string) for dict_string in tqdm.tqdm(data)]

        print('stage 6: writing to json (est: 10min)')
        with open(json_filename, 'w') as file:
            json.dump(data, file)
        print('complete')
    return None

In [3]:
directory = 'data'

optional_make_dir(directory)

urls = ['http://cseweb.ucsd.edu/~wckang/steam_reviews.json.gz', 
        'http://cseweb.ucsd.edu/~wckang/steam_games.json.gz']
for url in urls:
    download_json_gzip(url, directory)

for file in os.listdir(directory):
    if file.endswith('.gz'):
        gzip_to_json(file, directory)